# Step 1: Imports

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time;

# Step 2: Read images and initializations

In [ ]:
MIN_WIDTH = 0
MAX_WIDTH = 500
MIN_HEIGHT = 0
MAX_HEIGHT = 500
ACCEPTANCE_THRESHOLD = 0.7

t0 = time.time()

# simplesPic + simple1 & simple2 is another valid case

img = cv2.imread("./images/markers.png") # use aruco.jpg for more complex calculation

template1 = cv2.imread('./images/template1.png') 
template2 = cv2.imread('./images/template2.png')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Step 3: Define Template Matching function

In [ ]:
def templateMatch(img, warped_img, template, homography, color):
    # Rescale template to match the image size
    template = cv2.resize(template, (MAX_WIDTH - MIN_WIDTH, MAX_HEIGHT - MIN_HEIGHT), interpolation = cv2.INTER_AREA)

    img_gray = cv2.cvtColor(warped_img, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    res = cv2.matchTemplate(img_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    
    # if below the threshold, discard, else continue
    if res[0][0] < 0.8: 
        return
    else:
        print("Template matched with a value of:", res[0][0])
    
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    h, w = template_gray.shape[:2]
    top_left = max_loc
    top_right = (top_left[0] + w, top_left[1])
    bottom_left = (top_left[0], top_left[1] + h)
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # retransform homography transformed coordinates to the original image values
    og_pts = cv2.perspectiveTransform(np.array([[[top_left, top_right, bottom_left, bottom_right]]], dtype=np.float32).reshape(-1, 4, 2), np.linalg.inv(homography))[0]
    top_left_og = (int(og_pts[0][0]), int(og_pts[0][1]))
    top_right_og = (int(og_pts[1][0]), int(og_pts[1][1]))
    bottom_left_og = (int(og_pts[2][0]), int(og_pts[2][1]))
    bottom_right_og = (int(og_pts[3][0]), int(og_pts[3][1]))

    # draw marker borders
    cv2.line(img,top_left_og,top_right_og,color,2)
    cv2.line(img,top_right_og,bottom_right_og,color,2)
    cv2.line(img,bottom_right_og,bottom_left_og,color,2)
    cv2.line(img,bottom_left_og,top_left_og,color,2)

# Step 4: Define vector similarity comparison function

In [ ]:
# used to skip iterations and acelerate the matching process

def vector_similarity(v1, v2):
    v1 = [v1[0][0], v1[0][1]]
    v2 = [v2[0][0], v2[0][1]]

    vectorial_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    size_similarity = norm_v1 / norm_v2
    if (size_similarity < 0.95 or size_similarity > 1.05): # size acceptance threshold
        return False

    cosine = vectorial_product / (norm_v1 * norm_v2)
    if (cosine < 0.98): # 11deg acceptance threshold
        return False
    return True

# Step 5A: Regular Corner Detection(Harris & Shi-Tomasi)

In [ ]:
gray = np.float32(gray)

# maxCorners: more than 50 corners will take more than 12 minutes. Limit to preserve time usage.
# qualityLevel: 0.01 minimum. Corner acceptance ratio considering quality. Increasing the value would skip important corners
# minDistance: avoid corners being marked in the same spot(a few pixels to the side)
# blockSize: corner pixel block considered group size
# useHarrisDetector: True to Harris, False to Shi-Tomasi
corners = cv2.goodFeaturesToTrack(gray, maxCorners = 50, qualityLevel = 0.05, minDistance = 10, blockSize = 2, useHarrisDetector = True)
corners = np.int0(corners)

# Step 5B: Contour based Corner Detection(Faster)

In [ ]:
corners = []

# Binary inv to catch template borders
_,  thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
# Find the contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through each contour and if it is a 4 sided polygon, add corners
for border in contours:
    # Approximates a polygonal curve. Goal is to find 4 sided polygon.
    approx = cv2.approxPolyDP(border, 0.01*cv2.arcLength(border, True), True)
    if len(approx) == 4:
        for c in approx:
            corners.append(c)

# Convert list into no array type
corners = np.array(corners)

# Step 6: Paint detected corners in blue

In [ ]:
print("Number of corners:", len(corners))

for c in range(0, len(corners)):
    x = corners[c][0][0]
    y = corners[c][0][1]
    for i in range(-1, 2, 1):
        for j in range(-1, 2, 1):
            img[y + i][x + j] = (255, 0 , 0)

# Step 7: Iterate through all the corners and try to find a match with the templates

In [ ]:
# Generate all possible permutations of 4 corners
corner_permutations = itertools.permutations(range(len(corners)), 4)

count = 0
# Iterate over all the corner permutations
for indexes in corner_permutations:
    # Calculation status information. Prints every 100 iterations
    count += 1
    if (count % 10000) == 0:
        ts = time.time() - t0
        print("Count:", count, "Elapsed time:", ts)

    # Extract the x, y positions of the corners
    pts = corners[list(indexes)]   
    # Create 2 vectors from the 4 points to evaluate parallelism(Necessary to be a square)  
    v1 = pts[1] - pts[0]
    v2 = pts[2] - pts[3]
    # If not, discard
    if (not vector_similarity(v1, v2)):
        continue

    # Homography calculation and image transformation to match template. Evaluate similarity. If it matches paint the borders in red for 1st and green for 2nd
    homography, _ = cv2.findHomography(pts, np.array([[MIN_WIDTH, MIN_HEIGHT], [MIN_WIDTH, MAX_HEIGHT], [MAX_WIDTH, MAX_HEIGHT], [MAX_WIDTH, MIN_HEIGHT]], dtype=np.float32))
    warped_img = cv2.warpPerspective(img, homography, (MAX_WIDTH - MIN_WIDTH, MAX_HEIGHT - MIN_HEIGHT))
    templateMatch(img, warped_img, template1, homography, (0, 0, 255))
    templateMatch(img, warped_img, template2, homography, (0, 255, 0))

# Step 8: Show the result

In [ ]:
# Display the result
cv2.imshow("Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()